# MaxwellEvol: Solving Maxwell's Equations in the Einstein Toolkit with the Method of Lines

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

### NRPy+ Source Code for this module: [Maxwell/MaxwellCartesian_Evol.py](../edit/Maxwell/MaxwellCartesian_Evol.py) [\[tutorial\]](Tutorial-MaxwellCartesian.ipynb) Constructs Maxwell's equations and initial data as SymPy expressions

## Introduction:
This module focuses on using the equations developed in the [Tutorial-MaxwellCartesian](Tutorial-MaxwellCartesian.ipynb) tutorial module to build an Einstein Toolkit (ETK) thorn to solve Maxwell's equations in Cartesian coordinates. This tutorial will focus on implementing the time evolution aspects; the next will construct the thorn that will set up the initial data to be evolved.

When interfaced properly with the ETK, this module will propagate the initial data for $E_i$, $A_i$, and $\psi$ ( and $\Gamma$, if we so choose), defined in the next tutorial, forward in time by integrating the equations for $\partial_t E_i$, $\partial_t A_i$ and $\partial_t \psi$ (and possibly $\partial_t \Gamma$) subject to spatial boundary conditions. The time evolution itself is handled by the $\text{MoL}$ (Method of Lines) thorn in the $\text{CactusNumerical}$ arrangement, and the boundary conditions by the $\text{Boundary}$ thorn in the $\text{CactusBase}$ arrangement. 

Similar to the Maxwell initial data module, we will construct the WaveToyNRPy module in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the evolution equations into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2](#etk): Interfacing with the Einstein Toolkit
    1. [Step 2.a](#etkc): Constructing the Einstein Toolkit C-code calling functions that include the C code kernels
    1. [Step 2.b](#cclfiles): CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure
    1. [Step 2.c](#etk_list): Add the C file to Einstein Toolkit compilation list
1. [Step 3](#code_validation): Code Validation (To be performed)
1. [Step 4](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file
    


<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+:

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop


# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1c: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module.
import Maxwell.MaxwellCartesian_Evol as mwrhs
par.set_parval_from_str("System_to_use","System_I")
mwrhs.MaxwellCartesian_Evol()
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",2)

# Step 2: Register gridfunctions so they can be written to by NRPy.
# System I:
AIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","AIrhsD")
EIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","EIrhsD")
psiIrhs = gri.register_gridfunctions("EVOL","psiIrhs")

# Step 3: Set the rhs gridfunctions to their variables 
#         defined by MaxwellEvol().
for i in range(DIM):
    AIrhsD[i] = mwrhs.ArhsD[i]
    EIrhsD[i] = mwrhs.ErhsD[i]
psiIrhs = mwrhs.psi_rhs
Maxwell_Evol_to_printI = [\
                         lhrh(lhs=gri.gfaccess("out_gfs","AIrhsD0"),rhs=AIrhsD[0]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","AIrhsD1"),rhs=AIrhsD[1]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","AIrhsD2"),rhs=AIrhsD[2]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","EIrhsD0"),rhs=EIrhsD[0]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","EIrhsD1"),rhs=EIrhsD[1]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","EIrhsD2"),rhs=EIrhsD[2]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","psiIrhs"),rhs=psiIrhs)]
# Set outCverbose=False to avoid enormous file output.
Maxwell_Evol_CcodeKernelI = fin.FD_outputC("returnstring",Maxwell_Evol_to_printI,
                                           params="outCverbose=False").replace("AD","AID")\
                                                                      .replace("ED","EID")\
                                                                      .replace("psi","psiI")
# For debugging only:
#fin.FD_outputC("stdout",Maxwell_Evol_to_printI)

gri.glb_gridfcs_list = []
# Step 1c: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module.
par.set_parval_from_str("System_to_use","System_II")
mwrhs.MaxwellCartesian_Evol()
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",2)

# Step 2: Register gridfunctions so they can be written to by NRPy.
# System II:
AIIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","AIIrhsD")
EIIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","EIIrhsD")
psiIIrhs = gri.register_gridfunctions("EVOL","psiIIrhs")
Gammarhs = gri.register_gridfunctions("EVOL","Gammarhs")
# Step 3: Set the rhs gridfunctions to their variables 
#         defined by MaxwellEvol().

for i in range(DIM):
    AIIrhsD[i] = mwrhs.ArhsD[i]
    EIIrhsD[i] = mwrhs.ErhsD[i]
psiIIrhs = mwrhs.psi_rhs
Gammarhs = mwrhs.Gamma_rhs

# Step 4: Create the C code output kernel.
Maxwell_Evol_to_printII = [\
                          lhrh(lhs=gri.gfaccess("out_gfs","AIIrhsD0"),rhs=AIIrhsD[0]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","AIIrhsD1"),rhs=AIIrhsD[1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","AIIrhsD2"),rhs=AIIrhsD[2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","EIIrhsD0"),rhs=EIIrhsD[0]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","EIIrhsD1"),rhs=EIIrhsD[1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","EIIrhsD2"),rhs=EIIrhsD[2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","psiIIrhs"),rhs=psiIIrhs),\
                          lhrh(lhs=gri.gfaccess("out_gfs","Gammarhs"),rhs=Gammarhs)]
Maxwell_Evol_CcodeKernelII = fin.FD_outputC("returnstring",Maxwell_Evol_to_printII).replace("AD","AIID")\
                                                                                   .replace("ED","EIID")\
                                                                                   .replace("psiGF","psiIIGF")

Maxwell_Evol_looped = loop.loop(["i2","i1","i0"],["cctk_nghostzones[2]","cctk_nghostzones[1]","cctk_nghostzones[0]"],\
                               ["cctk_lsh[2]-cctk_nghostzones[2]","cctk_lsh[1]-cctk_nghostzones[1]",\
                                "cctk_lsh[0]-cctk_nghostzones[0]"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",\
                               (Maxwell_Evol_CcodeKernelI+Maxwell_Evol_CcodeKernelII).replace("time","cctk_time"))

# Step 5: Create directories for the thorn if they don't exist.
!mkdir MaxwellEvol     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir MaxwellEvol/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 6: Write the C code kernel to file.
with open("MaxwellEvol/src/Maxwell_Evol.h", "w") as file:
    file.write(str(Maxwell_Evol_looped))

with open("MaxwellEvol/src/NRPy_params.h", "w") as file:
    file.write("#define FD_CENTDERIVS_ORDER "+str(par.parval_from_str("finite_difference::FD_CENTDERIVS_ORDER"))+"\n")
    
# Step 7: Create a C code kernel to evaluate the constraint violation
Cviola_CcodeKernel = fin.FD_outputC("returnstring",[lhrh(lhs=gri.gfaccess("out_gfs","Cviola"),rhs=mwrhs.Cviola)])
Cviola_CcodeKernel_geminated = (Cviola_CcodeKernel.replace("ED","EID").replace("Cviola","CviolaI") + \
                                Cviola_CcodeKernel.replace("ED","EIID").replace("Cviola","CviolaII"))
Cviola_CcodeKernel_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                                     ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                     Cviola_CcodeKernel_geminated.replace("time","cctk_time"))
with open("MaxwellEvol/src/Constraint_violation.h", "w") as file:
    file.write(str(Cviola_CcodeKernel_looped))

<a id='etk'></a>

# Step 2: Interfacing with the Einstein Toolkit \[Back to [top](#toc)\]
$$\label{etk}$$



<a id='etkc'></a>

## Step 2.a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels \[Back to [top](#toc)\]
$$\label{etkc}$$

Now that we have generated the C code kernel *ScalarWave_RHSs.h* and the parameters file *NRPy_params.h*, we will need to write C code to make use of these files. To do this, we can simply follow the example within the [IDScalarWaveNRPy tutorial module](Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb). Functions defined by these files will be called by the Einstein Toolkit scheduler (specified in schedule.ccl below).

In [2]:
%%writefile MaxwellEvol/src/MaxwellEvol.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

#include "NRPy_params.h"

void MaxwellEvol_calc_constraint_violation(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  const CCTK_REAL *gammaDD00GF = gxx;
  const CCTK_REAL *gammaDD01GF = gxy;
  const CCTK_REAL *gammaDD02GF = gxz;
  const CCTK_REAL *gammaDD11GF = gyy;
  const CCTK_REAL *gammaDD12GF = gyz;
  const CCTK_REAL *gammaDD22GF = gzz;

    #include "Constraint_violation.h"
}

void MaxwellEvol_set_rhs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  const CCTK_REAL *gammaDD00GF = gxx;
  const CCTK_REAL *gammaDD01GF = gxy;
  const CCTK_REAL *gammaDD02GF = gxz;
  const CCTK_REAL *gammaDD11GF = gyy;
  const CCTK_REAL *gammaDD12GF = gyz;
  const CCTK_REAL *gammaDD22GF = gzz;
  
#include "Maxwell_Evol.h" 
}

/* Boundary Condition code adapted from WaveToyC thorn in ETK, implementing built-in
 * ETK BC functionality
 */
void MaxwellEvol_SelectBCs(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const char *bctype;


  bctype = NULL;
  if (CCTK_EQUALS(bound,"flat") || CCTK_EQUALS(bound,"static") ||
      CCTK_EQUALS(bound,"radiation") || CCTK_EQUALS(bound,"robin") ||
      CCTK_EQUALS(bound,"none"))
  {
    bctype = bound;
  }
  else if (CCTK_EQUALS(bound,"zero"))
  {
    bctype = "scalar";
  }

  /* Uses all default arguments, so invalid table handle -1 can be passed */
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::AID0GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::AID1GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::AID2GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::EID0GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::EID1GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::EID2GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::psiIGF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::AIID0GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::AIID1GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::AIID2GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::EIID0GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::EIID1GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::EIID2GF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::psiIIGF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "MaxwellEvol::GammaGF", bctype) < 0)
  {
    CCTK_WARN (0, "MaxwellEvol_Boundaries: Error selecting boundary condition");
  }
}

void MaxwellEvol_InitSymBound(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
      
  int sym[3];

  sym[0] = 1;
  sym[1] = 1;
  sym[2] = 1;

  SetCartSymVN(cctkGH, sym,"MaxwellEvol::AID0GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::AID1GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::AID2GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::EID0GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::EID1GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::EID2GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::psiIGF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::CviolaIGF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::AIID0GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::AIID1GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::AIID2GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::EIID0GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::EIID1GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::EIID2GF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::psiIIGF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::GammaGF");
  SetCartSymVN(cctkGH, sym,"MaxwellEvol::CviolaIIGF");

  return;
}

void MaxwellEvol_RegisterVars(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  CCTK_INT ierr CCTK_ATTRIBUTE_UNUSED = 0;
  /* Register all the evolved grid functions with MoL */
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::AID0GF"),  CCTK_VarIndex("MaxwellEvol::AIrhsD0GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::AID1GF"),  CCTK_VarIndex("MaxwellEvol::AIrhsD1GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::AID2GF"),  CCTK_VarIndex("MaxwellEvol::AIrhsD2GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::EID0GF"),  CCTK_VarIndex("MaxwellEvol::EIrhsD0GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::EID1GF"),  CCTK_VarIndex("MaxwellEvol::EIrhsD1GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::EID2GF"),  CCTK_VarIndex("MaxwellEvol::EIrhsD2GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::psiIGF"),  CCTK_VarIndex("MaxwellEvol::psiIrhsGF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::AIID0GF"),  CCTK_VarIndex("MaxwellEvol::AIIrhsD0GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::AIID1GF"),  CCTK_VarIndex("MaxwellEvol::AIIrhsD1GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::AIID2GF"),  CCTK_VarIndex("MaxwellEvol::AIIrhsD2GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::EIID0GF"),  CCTK_VarIndex("MaxwellEvol::EIIrhsD0GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::EIID1GF"),  CCTK_VarIndex("MaxwellEvol::EIIrhsD1GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::EIID2GF"),  CCTK_VarIndex("MaxwellEvol::EIIrhsD2GF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("MaxwellEvol::psiIIGF"),  CCTK_VarIndex("MaxwellEvol::psiIIrhsGF"));
  /* Register all the evolved Array functions with MoL */
  return;
}

Overwriting MaxwellEvol/src/MaxwellEvol.c


<a id='cclfiles'></a>

## Step 2.b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure \[Back to [top](#toc)\]
$$\label{cclfiles}$$

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the gridfunctions $A_i$, $E_i$, $\psi$, and $\Gamma$ to be visible to other thorns by using the keyword "public". 

In [3]:
%%writefile MaxwellEvol/interface.ccl
implements: MaxwellEvol

inherits: admbase Boundary grid

USES INCLUDE: loopcontrol.h
USES INCLUDE: Symmetry.h
USES INCLUDE: Boundary.h
    
CCTK_INT FUNCTION MoLRegisterEvolved(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
USES FUNCTION MoLRegisterEvolved

CCTK_INT FUNCTION GetBoundarySpecification(CCTK_INT IN size, CCTK_INT OUT ARRAY nboundaryzones, CCTK_INT OUT ARRAY is_internal, CCTK_INT OUT ARRAY is_staggered, CCTK_INT OUT ARRAY shiftout)
USES FUNCTION GetBoundarySpecification

CCTK_INT FUNCTION SymmetryTableHandleForGrid(CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION SymmetryTableHandleForGrid

CCTK_INT FUNCTION Boundary_SelectGroupForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN group_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectGroupForBC

CCTK_INT FUNCTION Boundary_SelectVarForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN var_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectVarForBC

public:
cctk_real system_I_rhs type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  AIrhsD0GF,AIrhsD1GF,AIrhsD2GF,EIrhsD0GF,EIrhsD1GF,EIrhsD2GF,psiIrhsGF
} "The evolved scalar fields"

public:
cctk_real system_II_rhs type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  AIIrhsD0GF,AIIrhsD1GF,AIIrhsD2GF,EIIrhsD0GF,EIIrhsD1GF,EIIrhsD2GF,psiIIrhsGF,GammarhsGF
} "The evolved scalar fields"

public:
cctk_real system_I type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  AID0GF,AID1GF,AID2GF,EID0GF,EID1GF,EID2GF,psiIGF
} "The evolved scalar fields"

public:
cctk_real system_II type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  AIID0GF,AIID1GF,AIID2GF,EIID0GF,EIID1GF,EIID2GF,psiIIGF,GammaGF
} "The evolved scalar fields"

public:
cctk_real system_I_constraint_violation type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  CviolaIGF
} "The constraint violation for system I"

public:
cctk_real system_II_constraint_violation type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  CviolaIIGF
} "The constraint violation for system II"


Overwriting MaxwellEvol/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). A number of parameters are defined, and more parameters can be easily added in later versions. We also set the number of timelevels we will store in memory.

In [4]:
%%writefile MaxwellEvol/param.ccl
shares: MethodOfLines

USES CCTK_INT MoL_Num_Evolved_Vars
USES CCTK_INT MoL_Num_ArrayEvolved_Vars

restricted:
CCTK_INT MaxwellEvol_MaxNumEvolvedVars "Number of evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_Evolved_Vars STEERABLE=RECOVER
{
  15:15 :: "Number of evolved variables used by this thorn"
} 15

restricted:
CCTK_INT SimpleWave_MaxNumArrayEvolvedVars "Number of Array evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_ArrayEvolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of Array evolved variables used by this thorn"
} 0

restricted:
KEYWORD bound "Type of boundary condition to use"
{
  "flat"      :: "Flat (von Neumann, n grad phi = 0) boundary condition"
  "static"    :: "Static (Dirichlet, dphi/dt=0) boundary condition"
  "radiation" :: "Radiation boundary condition"
  "robin"     :: "Robin (phi(r) = C/r) boundary condition"
  "zero"      :: "Zero (Dirichlet, phi=0) boundary condition"
  "none"      :: "Apply no boundary condition"
} "static"

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_REAL amp "The amplitude of the wavepacket"
{
 *:* :: "The amplitude of the wavepacket"
} 1.0

restricted:
CCTK_REAL lam "The size lambda of the wavepacket"
{
 *:* :: "The size lambda of the wavepacket"
} 1.0

Overwriting MaxwellEvol/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We first assign storage for both scalar gridfunctions, and then specify the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [5]:
%%writefile MaxwellEvol/schedule.ccl
STORAGE: system_I_rhs[timelevels]
STORAGE: system_I[timelevels]
STORAGE: system_II_rhs[timelevels]
STORAGE: system_II[timelevels]
STORAGE: system_I_constraint_violation[timelevels]
STORAGE: system_II_constraint_violation[timelevels]

schedule MaxwellEvol_InitSymBound at BASEGRID
{
  LANG: C
  OPTIONS: global
} "Schedule symmetries"

schedule MaxwellEvol_calc_constraint_violation IN CCTK_ANALYSIS
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: EID0GF(Everywhere)
  READS: EID1GF(Everywhere)
  READS: EID2GF(Everywhere)
  READS: EIID0GF(Everywhere)
  READS: EIID1GF(Everywhere)
  READS: EIID2GF(Everywhere)
  WRITES: CviolaIGF(Interior)
  WRITES: CviolaIIGF(Interior)
}"Calculate the contraint violation of the simulation"

schedule MaxwellEvol_set_rhs as WaveToy_Evolution IN MoL_CalcRHS
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: AID0GF(Everywhere)
  READS: AID1GF(Everywhere)
  READS: AID2GF(Everywhere)
  READS: EID0GF(Everywhere)
  READS: EID1GF(Everywhere)
  READS: EID2GF(Everywhere)
  READS: psiIGF(Everywhere)
  READS: AIID0GF(Everywhere)
  READS: AIID1GF(Everywhere)
  READS: AIID2GF(Everywhere)
  READS: EIID0GF(Everywhere)
  READS: EIID1GF(Everywhere)
  READS: EIID2GF(Everywhere)
  READS: psiIIGF(Everywhere)
  READS: GammaGF(Everywhere)
  WRITES: AIrhsD0GF(Interior)
  WRITES: AIrhsD1GF(Interior)
  WRITES: AIrhsD2GF(Interior)
  WRITES: EIrhsD0GF(Interior)
  WRITES: EIrhsD1GF(Interior)
  WRITES: EIrhsD2GF(Interior)
  WRITES: psiIrhsGF(Interior)
  WRITES: AIIrhsD0GF(Interior)
  WRITES: AIIrhsD1GF(Interior)
  WRITES: AIIrhsD2GF(Interior)
  WRITES: EIIrhsD0GF(Interior)
  WRITES: EIIrhsD1GF(Interior)
  WRITES: EIIrhsD2GF(Interior)
  WRITES: psiIIrhsGF(Interior)
  WRITES: GammarhsGF(Interior)
} "Evolution of Maxwell's equations"

schedule MaxwellEvol_SelectBCs in MoL_PostStep
{
  LANG: C
  OPTIONS: level
  SYNC: system_I
  SYNC: system_II
} "Boundaries of Maxwell's equations"

schedule GROUP ApplyBCs as MaxwellEvol_ApplyBCs in MoL_PostStep after MaxwellEvol_SelectBCs
{
} "Apply boundary conditions"


schedule GROUP ApplyBCs as MaxwellEvol_ApplyBCs at POSTRESTRICT
{
} "Apply boundary conditions"

schedule MaxwellEvol_RegisterVars in MoL_Register
{
  LANG: C
  OPTIONS: meta
} "Register Variables for MoL"


Overwriting MaxwellEvol/schedule.ccl


<a id='etk_list'></a>

## Step 2.c: Add the C file to Einstein Toolkit compilation list \[Back to [top](#toc)\]
$$\label{etk_list}$$

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [6]:
%%writefile MaxwellEvol/src/make.code.defn
SRCS = MaxwellEvol.c

Overwriting MaxwellEvol/src/make.code.defn


<a id='code_validation'></a>

# Step 3: Code Validation (To be performed) \[Back to [top](#toc)\]
$$\label{code_validation}$$

<a id='latex_pdf_output'></a>

# Step 4: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ETK_thorn-MaxwellEvol.pdf](Tutorial-ETK_thorn-MaxwellEvol.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ETK_thorn-MaxwellEvol.ipynb
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-MaxwellEvol.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-MaxwellEvol.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-MaxwellEvol.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ETK_thorn-MaxwellEvol.ipynb to latex
[NbConvertApp] Writing 95772 bytes to Tutorial-ETK_thorn-MaxwellEvol.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
